<a href="https://colab.research.google.com/github/yunuxemre/test/blob/master/SPT3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install visions==0.7.4
!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR

In [ ]:
data = pd.read_csv('/content/StudentsPerformance.csv')

In [ ]:
data

In [ ]:
encoder = LabelEncoder()
data['gender'] = encoder.fit_transform(data['gender'])
data['race/ethnicity'] = encoder.fit_transform(data['race/ethnicity'])
data['parental level of education'] = encoder.fit_transform(data['parental level of education'])
data['lunch'] = encoder.fit_transform(data['lunch'])
data['test preparation course'] = encoder.fit_transform(data['test preparation course'])

In [ ]:
report = ProfileReport(data)

In [ ]:
report

In [ ]:
X = data[['math score', 'reading score', 'writing score']]
y_math = X['math score']
y_reading = X['reading score']
y_writing = X['writing score']

In [ ]:
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

In [ ]:
X_math_train, X_math_test, y_math_train, y_math_test = train_test_split(X, y_math, train_size=0.7)
X_reading_train, X_reading_test, y_reading_train, y_reading_test = train_test_split(X, y_reading, train_size=0.7)
X_writing_train, X_writing_test, y_writing_train, y_writing_test = train_test_split(X, y_writing, train_size=0.7)

In [ ]:
math_model = LinearRegression()
reading_model = LinearRegression()
writing_model = LinearRegression()

In [ ]:
def cross_val_and_overfitting_analysis(model, X_train, y_train, folds=5):
    # Cross-validation
    cv_scores = cross_val_score(model, X_train, y_train, cv=folds)

    # Fit the model to the entire training set to analyze overfitting
    model.fit(X_train, y_train)
    train_score = model.score(X_train, y_train)

    # Print results
    print(f'Cross-Validation Scores: {cv_scores}')
    print(f'Mean CV Score: {np.mean(cv_scores)}')
    print(f'Training Score: {train_score}')
    print(f'Overfitting: {train_score - np.mean(cv_scores)}')

In [ ]:
print("Math Model:")
cross_val_and_overfitting_analysis(math_model, X_math_train, y_math_train)
print("\nReading Model:")
cross_val_and_overfitting_analysis(reading_model, X_reading_train, y_reading_train)
print("\nWriting Model:")
cross_val_and_overfitting_analysis(writing_model, X_writing_train, y_writing_train)

In [ ]:
knn_model = KNeighborsRegressor()
print("\nK-Nearest Neighbors Model:")
cross_val_and_overfitting_analysis(knn_model, X_math_train, y_math_train)

In [ ]:
tree_model = DecisionTreeRegressor()
print("\nDecision Tree Model:")
cross_val_and_overfitting_analysis(tree_model, X_math_train, y_math_train)

In [ ]:
svm_model = SVR()
print("\nSupport Vector Machine Model:")
cross_val_and_overfitting_analysis(svm_model, X_math_train, y_math_train)

In [16]:
models = {
    "Linear Regression": LinearRegression(),
    "K-Nearest Neighbors": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Support Vector Machine": SVR(),
}

In [18]:
for name, model in models.items():
    if "Classifier" in name:
        y_train = (y_math_train >= 50).astype(int)  # Assuming classification task for KNeighborsClassifier and DecisionTreeClassifier
    else:
        y_train = y_math_train  # Assuming regression task for other models

    model.fit(X_math_train, y_train)
    print(f"{name} trained.")

Linear Regression trained.
K-Nearest Neighbors trained.
Decision Tree trained.
Support Vector Machine trained.


In [22]:
for name, model in models.items():
    if "Classifier" in name:
        y_test_pred = (model.predict(X_math_test) >= 0.5).astype(int)  # Assuming classification task for KNeighborsClassifier and DecisionTreeClassifier
    else:
        y_test_pred = model.predict(X_math_test)  # Assuming regression task for other models

    accuracy = model.score(X_math_test, y_math_test) * 100
    print(f"{name}: {accuracy:.2f}%")


Linear Regression: 100.00%
K-Nearest Neighbors: 99.35%
Decision Tree: 100.00%
Support Vector Machine: 96.52%
